In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CubicSpline
import yfinance as yf
from datetime import datetime,time, timedelta
from scipy.stats import norm, kurtosis, skew
from statistics import mean, stdev
import xlsxwriter

In [2]:
#Fetching the Downloaded Options Data

options_df = pd.read_csv("2018_1MOptionsData.csv")
options_df['strike_price'] = options_df['strike_price']/1000
options_df['date'] = pd.to_datetime(options_df['date']).dt.date
options_df['exdate'] = pd.to_datetime(options_df['exdate']).dt.date
options_df['Maturity'] = options_df['exdate']-options_df['date']
options_df['Maturity'] = options_df['Maturity'].dt.days.astype('int16')

/var/folders/cp/p3rk934j69l8v61fbfgp2c5r0000gn/T/ipykernel_1680/3492196636.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  options_df = pd.read_csv("2018_1MOptionsData.csv")


In [3]:
#Getting the list of Tickers
tickers = sorted(list(options_df['ticker'].unique()))

In [5]:
#Stock Data which has both the WRDS and Yahoo Finance DAta
stock_df = pd.read_csv("Ticker100_StockData-NeW.csv")

In [6]:
#Filtering the CUSIP which has same but change in the Ticker
stock_hash = {}
test_df = stock_df[['TICKER','CUSIP']].reset_index()[['TICKER','CUSIP']].drop_duplicates()

for index in test_df.index:
    stock_hash[test_df.loc[index]['CUSIP']] = test_df.loc[index]['TICKER']

stock_hash_df = pd.DataFrame(stock_hash,index=['TICKER']).T 
stock_hash_df['CUSIP'] = stock_hash_df.index
stock_df = stock_df.merge(stock_hash_df,on="CUSIP",how = 'left')
stock_df.columns = ['PERMNO', 'date', 'TICKER_OLD', 'CUSIP', 'PRC', 'TICKER']
stock_df = stock_df[['PERMNO', 'date', 'TICKER', 'CUSIP', 'PRC']]
stock_df = stock_df.set_index(pd.to_datetime(stock_df['date']))

In [7]:
#Display to check on the data:
stock_df.head()

,PERMNO,date,TICKER,CUSIP,PRC
date,,,,,
2018-01-02,10104,1/2/18,ORCL,68389X10,46.63
2018-01-03,10104,1/3/18,ORCL,68389X10,47.71
2018-01-04,10104,1/4/18,ORCL,68389X10,48.18
2018-01-05,10104,1/5/18,ORCL,68389X10,48.47
2018-01-08,10104,1/8/18,ORCL,68389X10,48.98


In [8]:
#Comparing the data. 
set(tickers) - set(stock_df['TICKER'].unique())

set()

In [9]:
#Getting the Treasury data which helps to caluculate BlackScholes Pricing:
treasury_df = pd.read_excel("Treasury_Data_Cleaned.xlsx")
treasury_df.index = treasury_df['Date']
treasury_df = pd.DataFrame(treasury_df['Close'])

In [10]:
#BlackScholes Model Function:
def black_scholes_call_price(S, K, T, r, q, sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    return call_price

In [11]:
#Methods of Moments, First 4 moments are providing using this:
def moments(call_price_second_derivate):
  skewness_value = skew(call_price_second_derivate,axis = 0, bias = True)
  kurtosis_value = kurtosis(call_price_second_derivate,axis = 0, fisher = True, bias = True)
  return mean(call_price_second_derivate),stdev(call_price_second_derivate),skewness_value,kurtosis_value

In [12]:
#Generating the Specific Option and Stock for a Maturity:
def generate_pdf(options_df,stock_df,treasury_df,date):

    date = pd.to_datetime(date).date()
    stock_date = datetime.strptime(str(date), "%Y-%m-%d").strftime("%-m/%d/%y")
    
    reference_maturity  = 30
    sample_df = options_df[options_df['date'] == date].dropna()
    sample_df = sample_df.sort_values(by='date')
    maturities = list(set(sample_df.Maturity))
    
  
    nearest_maturity = ([abs(maturity - reference_maturity) for maturity in maturities])
    reference_maturity = nearest_maturity.index(min(nearest_maturity))
    sample_df = sample_df[sample_df['Maturity'] == maturities[reference_maturity] ]
    maturity = maturities[reference_maturity]
    
    while len(sample_df) < 2:
        maturities = list(set(maturities) - set([maturity]))
        if len(maturities) > 1:
            nearest_maturity = ([abs(maturity - reference_maturity) for maturity in maturities])
            reference_maturity = nearest_maturity.index(min(nearest_maturity))
            maturity = maturities[reference_maturity]
        else:
            if len(maturities):
                maturity = maturities[0]
            else:
                return maturity,[0],[0],[0],[0],[0],[0]
            
        sample_df = options_df[options_df['date'] == date].dropna()
        sample_df = sample_df.sort_values(by='date')

        sample_df = sample_df[sample_df['Maturity'] == maturity ]
        
        
    
    
    sample_df = sample_df.sort_values("strike_price")
    sample_df['Call Price'] = (sample_df['best_bid']+sample_df['best_offer'])/2

    stock_price_bk = stock_df.loc[stock_date]['PRC']
    strike_prices = sorted(list(sample_df['strike_price']))
    imp_vol = sorted(list(sample_df['impl_volatility']))
    
                     
                     
   
    strike_at_the_money_min = ([abs(strike - stock_price_bk) for strike in strike_prices])
    at_the_money_index = strike_at_the_money_min.index(min(strike_at_the_money_min))

    std_index = int(len(strike_prices)/6)
    strike_price_spline_index = [0,at_the_money_index,len(strike_prices)-1]

    for i in range(1,3):
        prev_index_strike = at_the_money_index-(i*std_index)
        next_index_strike = at_the_money_index+(i*std_index)
        if prev_index_strike > 0: strike_price_spline_index.append(prev_index_strike)
        if next_index_strike < len(strike_prices)-1: strike_price_spline_index.append(next_index_strike)

    strike_price_spline_index = sorted(list(set(strike_price_spline_index)))
    strike_prices_spline = list(sample_df.iloc[strike_price_spline_index]['strike_price'])
    imp_vol = list(sample_df.iloc[strike_price_spline_index]['impl_volatility'])
  
    print(strike_prices_spline)
    print(imp_vol)
    if len(strike_prices_spline) < 2:
        return maturity,[0],[0],[0],[0],[0],[0]
                     
  
    # Create a cubic spline interpolation of the data
    imp_vol_smooth = CubicSpline(strike_prices_spline, imp_vol)

    
    strike_prices_bk = np.linspace(strike_prices[0],strike_prices[-1], num=len(strike_prices)*2)
    time_bk =  list(sample_df['Maturity'].unique())[0]/365
    
    treasury_date = str(datetime.strptime(str(date), "%Y-%m-%d") + timedelta(days=maturity)).split()[0]
    rate_bk = treasury_df.loc[treasury_date]['Close']/100
    
    call_price_bk = []

    #blackscholes_prices:
    for _,strike_price in enumerate(strike_prices_bk):
        call_price_bk.append(black_scholes_call_price(stock_price_bk,strike_price,time_bk,rate_bk,0,imp_vol_smooth(strike_price)))

    #return call_price_bk

    dk = 10
    call_price_second_derivate = []
    strike_derivative = np.linspace(strike_prices[0],strike_prices[-1], num=int((strike_prices[-1]-strike_prices[0])/dk))
    for i,strike_price in enumerate(strike_derivative[1:-1]):
        fxdkPrev = black_scholes_call_price(stock_price_bk,strike_derivative[i],time_bk,rate_bk,0,imp_vol_smooth(strike_derivative[i]))
        fxdkNext = black_scholes_call_price(stock_price_bk,strike_derivative[i+2],time_bk,rate_bk,0,imp_vol_smooth(strike_derivative[i+2]))
        fx= black_scholes_call_price(stock_price_bk,strike_price,time_bk,rate_bk,0,imp_vol_smooth(strike_price))

        call_price_second_derivate.append((np.exp(-rate_bk*time_bk))*(fxdkPrev+fxdkNext-(2*fx))/(dk**2))

    return maturity,strike_prices,sample_df['Call Price'],strike_prices_bk,call_price_bk,strike_derivative[1:-1],call_price_second_derivate

In [13]:
#Get the List of the dates:
def get_dates(options_df,delay):
    date = "2018-01-02"
    current_date  = datetime.strptime(str(date), "%Y-%m-%d")
    forward_date  = current_date + timedelta(days = delay)

    current_date_index = current_date.strftime("%-m/%d/%y")
    dates_to_run = [current_date_index]
    unique_dates = options_df.date.unique()


    for i in range(0,400):
        if forward_date.date() in unique_dates:
            forward_date_index = forward_date.strftime("%-m/%d/%y")
            dates_to_run.append(forward_date_index)
            forward_date  = forward_date + timedelta(days = delay)  
        else:
            forward_date  = forward_date + timedelta(days = 1)
    
        if (forward_date >= datetime(2024, 1, 1, 0, 0)):
            break

    return dates_to_run

    

In [20]:
#Here we get the Moments data based on the specific data and Tickers:
def get_moments_data(option_df,stock_df,dates_to_run,tickers):
    moments_data = {}
    #tickers = ['META']
    for i,ticker in enumerate(tickers):
        option_ticker_df = options_df[options_df['ticker'] == ticker]
        cusip = list(option_ticker_df['cusip'].unique())[0]
        stock_ticker_df  = stock_df[(stock_df['TICKER'] == ticker) & (stock_df['CUSIP'] == cusip)]
        stock_ticker_index = stock_ticker_df.index
        option_ticker_df = option_ticker_df[option_ticker_df['cusip'] == cusip]

        for index,date in enumerate(dates_to_run):
            option_checker = option_ticker_df[option_ticker_df['date'] == pd.to_datetime(date).date()]
            maturity_checker = option_checker['Maturity'].unique()
            if date in stock_ticker_index and len(option_ticker_df[option_ticker_df['date'] == pd.to_datetime(date).date()].dropna()):
                print("***** Ticker : ",ticker," Date : ",date,"******")
                maturity,strike_prices,call_prices,strike_prices_bk,call_price_bk,strike_derivative,call_price_second_derivate = generate_pdf(option_ticker_df,stock_ticker_df,treasury_df,date)
                if len(call_price_second_derivate) > 2:    
                    mean_value,std_value,skew_value,kurtosis_value = moments(call_price_second_derivate)
                    hash_key = ticker+" "+date+" ("+str(maturity)+")"
                    moments_data[hash_key] = [mean_value,std_value,skew_value,kurtosis_value]
    

    moments_df = pd.DataFrame(moments_data).T
    moments_df.columns = ['mean','std','skew','kurtosis']
    moments_df['Date'] = [index.split()[1] for index in moments_df.index]
    moments_df['Ticker'] = [index.split()[0] for index in moments_df.index]
    
    
    return moments_df
        
    

In [21]:
#Caluculate Returns for Long. Short and Benchmark:
def get_return_data(intial_investment,backtest_df,tickers):
    shares = 0
    return_ticker_data = backtest_df[backtest_df['TICKER'].isin(tickers)]
    return_ticker_data.columns = ['PERMNO', 'DATE', 'TICKER','CUSIP','PRICE']
    return_ticker_data = return_ticker_data[['TICKER','PRICE']]
    return_ticker_data.reset_index(inplace=True)
    
    
    return_ticker_data = return_ticker_data.pivot(index="date",columns="TICKER", values="PRICE").rename_axis(columns=None

    return_ticker_data = return_ticker_data[return_ticker_data.columns[1:]]
    
    shares = list((1/return_ticker_data.iloc[0])*intial_investment)
    return_ticker_data = return_ticker_data*shares
    return_ticker_data['CurrentValue'] = return_ticker_data.sum(axis=1)
    
    return return_ticker_data

In [16]:
#Returns which is used on the caluculated: 
def returns(date,investment,moments_df,option_ticker_cusip):
    
    #Backtesting dataframe:
    current_date  = datetime.strptime(str(date), "%m/%d/%y")
    forward_date  = current_date + timedelta(days = 30)
    current_date_index = current_date.strftime("%-m/%d/%y")
    forward_date_index = forward_date.strftime("%-m/%d/%y")
    backtest_df = stock_df.loc[current_date_index:forward_date_index]

    
    #eturn forward_date,backtest_df
    forward_date = list(backtest_df[-1:].index)[0].date().strftime("%-m/%d/%y")
    
    current_moments_df = moments_df[moments_df['Date'] == date].sort_values("skew")
    short_portfolio = [tick.split()[0] for tick in list(current_moments_df.index)[:5]]
    long_portfolio  = [tick.split()[0] for tick in list(current_moments_df.index)[-5:]]

    benchmark_portfolio = short_portfolio+long_portfolio
    
    cusip_value = option_ticker_cusip[option_ticker_cusip['ticker'].isin(benchmark_portfolio)]['cusip']
    
    backtest_df = backtest_df[backtest_df['CUSIP'].isin(cusip_value)]

    long_portfolio_return = get_return_data(investment*2,backtest_df,long_portfolio).loc[forward_date]
    long_portfolio_return_2 = get_return_data(investment,backtest_df,long_portfolio).loc[forward_date]
    short_portfolio_return = get_return_data(investment,backtest_df,short_portfolio).loc[forward_date]
    benchmark_portfolio_return = get_return_data(investment,backtest_df,benchmark_portfolio).loc[forward_date]
    
    return long_portfolio,short_portfolio,forward_date_index,long_portfolio_return,long_portfolio_return_2,short_portfolio_return,benchmark_portfolio_return
    print(short_companies)

In [17]:
#Get Monthly and Weekly Dates:
dates_to_run_weekly = get_dates(options_df,7)
dates_to_run_monthly = get_dates(options_df,30)

In [23]:
moments_monthly_data.head()

,mean,std,skew,kurtosis,Date,Ticker
ABBV 2/01/18 (29),0.038531,0.022654,0.526264,-1.500000,2/01/18,ABBV
ABBV 4/04/18 (30),0.043343,0.028697,-0.160920,-1.500000,4/04/18,ABBV
ABBV 5/04/18 (28),0.028798,0.027625,0.077100,-1.617375,5/04/18,ABBV
ABBV 4/06/20 (32),0.031972,0.019972,-0.331601,-1.409909,4/06/20,ABBV
ABBV 5/06/20 (30),0.003194,0.128941,-0.656504,-1.500000,5/06/20,ABBV
...,...,...,...,...,...,...
XOM 10/07/22 (28),0.041820,0.015546,0.651600,-1.500000,10/07/22,XOM
XOM 11/07/22 (32),0.014641,0.017246,0.920633,-0.551065,11/07/22,XOM
XOM 12/07/22 (30),0.014441,0.018927,1.078202,-0.177659,12/07/22,XOM
XOM 1/06/23 (28),0.016838,0.021135,0.829143,-1.014785,1/06/23,XOM


In [22]:
moments_monthly_data = get_moments_data(options_df,stock_df,dates_to_run_monthly,tickers)

***** Ticker :  ABBV  Date :  1/02/18 ******
[85.0, 93.5, 96.0, 98.5, 102.0, 107.0, 110.0]
[0.393324, 0.270718, 0.271401, 0.264406, 0.26339, 0.262489, 0.267643]
***** Ticker :  ABBV  Date :  2/01/18 ******
[90.0, 100.0, 108.0, 116.0, 124.0, 132.0, 140.0]
[0.446113, 0.369883, 0.292437, 0.272009, 0.275909, 0.303772, 0.323783]
***** Ticker :  ABBV  Date :  3/05/18 ******
[95.0, 106.0, 111.0, 116.0, 121.0, 126.0, 140.0]
[0.467185, 0.344985, 0.324368, 0.294787, 0.287687, 0.294237, 0.380841]
***** Ticker :  ABBV  Date :  4/04/18 ******
[70.0, 88.0, 93.0, 98.0, 105.0, 127.0]
[0.748316, 0.43385, 0.385891, 0.366441, 0.377314, 0.460166]
***** Ticker :  ABBV  Date :  5/04/18 ******
[80.0, 91.0, 95.5, 100.0, 109.0, 118.0, 140.0]
[0.383711, 0.273937, 0.265895, 0.267452, 0.246886, 0.309645, 0.508426]
***** Ticker :  ABBV  Date :  6/04/18 ******
[85.0, 96.0, 99.0, 104.0, 110.0, 130.0]
[0.283631, 0.254319, 0.238165, 0.237157, 0.240718, 0.427289]
***** Ticker :  ABBV  Date :  7/05/18 ******
[80.0, 87.0

***** Ticker :  ABBV  Date :  8/08/22 ******
[75.0, 90.0, 115.0, 140.0, 165.0, 190.0, 225.0]
[0.935213, 0.682647, 0.427759, 0.242454, 0.306137, 0.507118, 0.72977]
***** Ticker :  ABBV  Date :  9/07/22 ******
[80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 205.0]
[0.802682, 0.441509, 0.327265, 0.235366, 0.32077, 0.514727, 0.711844]
***** Ticker :  ABBV  Date :  10/07/22 ******
[120.0, 133.0, 139.0, 145.0, 152.5, 215.0]
[0.39285, 0.336051, 0.321276, 0.300484, 0.286612, 0.759246]
***** Ticker :  ABBV  Date :  11/07/22 ******
[75.0, 132.0, 140.0, 148.0, 165.0, 190.0, 210.0]
[1.170673, 0.334472, 0.282054, 0.252184, 0.220269, 0.397407, 0.525996]
***** Ticker :  ABBV  Date :  12/07/22 ******
[147.0, 148.0, 155.0, 165.0, 175.0, 185.0, 210.0]
[0.260255, 0.270693, 0.237454, 0.217081, 0.200913, 0.241411, 0.363755]
***** Ticker :  ABBV  Date :  1/06/23 ******
[85.0, 95.0, 155.0, 167.5, 180.0, 192.5, 235.0]
[1.174027, 0.903312, 0.229685, 0.206127, 0.191303, 0.334342, 0.658315]
***** Ticker :  ABBV  Date 

***** Ticker :  ACN  Date :  2/07/22 ******
[245.0, 265.0, 305.0, 345.0, 385.0, 425.0, 495.0]
[0.408707, 0.437477, 0.38518, 0.272518, 0.299349, 0.445966, 0.675835]
***** Ticker :  ACN  Date :  3/09/22 ******
[200.0, 210.0, 270.0, 310.0, 350.0, 390.0, 480.0]
[0.772309, 0.73667, 0.52847, 0.413228, 0.374868, 0.480707, 0.818442]
***** Ticker :  ACN  Date :  4/08/22 ******
[285.0, 290.0, 315.0, 340.0, 365.0, 390.0, 460.0]
[0.375178, 0.366505, 0.324102, 0.25313, 0.241654, 0.302618, 0.630629]
***** Ticker :  ACN  Date :  5/09/22 ******
[200.0, 250.0, 285.0, 320.0, 355.0, 460.0]
[0.721253, 0.495302, 0.389746, 0.343679, 0.362652, 0.846619]
***** Ticker :  ACN  Date :  6/08/22 ******
[180.0, 230.0, 270.0, 305.0, 340.0, 375.0, 430.0]
[0.765645, 0.538926, 0.387612, 0.327314, 0.293557, 0.38499, 0.69111]
***** Ticker :  ACN  Date :  7/08/22 ******
[180.0, 210.0, 245.0, 280.0, 315.0, 350.0, 420.0]
[0.830696, 0.572061, 0.410022, 0.295897, 0.280701, 0.525568, 0.663562]
***** Ticker :  ACN  Date :  8/08

***** Ticker :  ADBE  Date :  6/04/21 ******
[245.0, 425.0, 470.0, 505.0, 545.0, 600.0]
[1.20676, 0.409665, 0.299256, 0.252079, 0.25878, 0.300376]
***** Ticker :  ADBE  Date :  7/06/21 ******
[365.0, 520.0, 565.0, 597.5, 650.0, 720.0]
[0.74695, 0.332593, 0.265015, 0.249509, 0.228309, 0.300897]
***** Ticker :  ADBE  Date :  8/05/21 ******
[385.0, 520.0, 580.0, 630.0, 680.0, 720.0]
[0.645638, 0.354723, 0.243883, 0.209779, 0.199343, 0.234418]
***** Ticker :  ADBE  Date :  9/07/21 ******
[335.0, 500.0, 590.0, 660.0, 730.0, 810.0]
[0.925681, 0.521694, 0.336044, 0.266912, 0.261517, 0.333048]
***** Ticker :  ADBE  Date :  10/07/21 ******
[350.0, 430.0, 505.0, 580.0, 655.0, 730.0, 880.0]
[0.554789, 0.524676, 0.369831, 0.253904, 0.26463, 0.352566, 0.558284]
***** Ticker :  ADBE  Date :  11/08/21 ******
[405.0, 560.0, 615.0, 665.0, 715.0, 780.0]
[0.759224, 0.385893, 0.287992, 0.252339, 0.249339, 0.304673]
***** Ticker :  ADBE  Date :  12/08/21 ******
[365.0, 480.0, 585.0, 655.0, 725.0, 850.0, 89

***** Ticker :  AVGO  Date :  9/04/20 ******
[170.0, 285.0, 330.0, 362.5, 395.0, 460.0, 495.0]
[1.12984, 0.608475, 0.484489, 0.425801, 0.410211, 0.504603, 0.572814]
***** Ticker :  AVGO  Date :  10/05/20 ******
[190.0, 300.0, 340.0, 365.0, 390.0, 435.0, 450.0]
[0.828484, 0.489585, 0.436, 0.393199, 0.34852, 0.356023, 0.36828]
***** Ticker :  AVGO  Date :  11/04/20 ******
[250.0, 310.0, 340.0, 365.0, 390.0, 430.0, 490.0]
[0.593101, 0.45729, 0.390695, 0.363483, 0.275511, 0.325575, 0.455431]
***** Ticker :  AVGO  Date :  12/04/20 ******
[295.0, 362.5, 385.0, 410.0, 455.0, 500.0, 520.0]
[0.421749, 0.409088, 0.375318, 0.33883, 0.371457, 0.399467, 0.409557]
***** Ticker :  AVGO  Date :  1/04/21 ******
[230.0, 340.0, 395.0, 425.0, 452.5, 505.0]
[0.95164, 0.491666, 0.36044, 0.31998, 0.290627, 0.350274]
***** Ticker :  AVGO  Date :  2/03/21 ******
[230.0, 390.0, 435.0, 465.0, 500.0, 550.0]
[1.200818, 0.475181, 0.366414, 0.309872, 0.26391, 0.358256]
***** Ticker :  AVGO  Date :  3/05/21 ******
[3

...

***** Ticker :  V  Date :  9/07/21 ******
[180.0, 195.0, 210.0, 225.0, 240.0, 255.0, 285.0]
[0.337483, 0.282257, 0.240083, 0.191802, 0.190484, 0.252706, 0.349614]
***** Ticker :  V  Date :  10/07/21 ******
[125.0, 180.0, 205.0, 230.0, 255.0, 280.0, 300.0]
[0.845653, 0.46695, 0.342377, 0.239924, 0.242984, 0.47071, 0.364947]
***** Ticker :  V  Date :  11/08/21 ******
[190.0, 205.0, 220.0, 235.0, 250.0, 280.0]
[0.322868, 0.24911, 0.232138, 0.237154, 0.259984, 0.53263]
***** Ticker :  V  Date :  12/08/21 ******
[135.0, 170.0, 190.0, 210.0, 230.0, 250.0, 270.0]
[0.75995, 0.450647, 0.327723, 0.259895, 0.256192, 0.306393, 0.474382]
***** Ticker :  V  Date :  1/07/22 ******
[135.0, 165.0, 190.0, 215.0, 240.0, 265.0, 295.0]
[0.908554, 0.581528, 0.410018, 0.280038, 0.282194, 0.337088, 0.453044]
***** Ticker :  V  Date :  2/07/22 ******
[190.0, 195.0, 210.0, 225.0, 240.0, 255.0, 295.0]
[0.33128, 0.330474, 0.305115, 0.259239, 0.241701, 0.246824, 0.365296]
***** Ticker :  V  Date :  3/09/22 ******


***** Ticker :  VRTX  Date :  3/05/21 ******
[105.0, 180.0, 197.5, 212.5, 270.0, 320.0]
[0.996734, 0.374204, 0.357286, 0.364804, 0.510995, 0.887141]
***** Ticker :  VRTX  Date :  4/05/21 ******
[110.0, 175.0, 197.5, 215.0, 232.5, 270.0]
[0.888272, 0.59564, 0.486932, 0.473611, 0.498361, 0.677077]
***** Ticker :  VRTX  Date :  5/05/21 ******
[110.0, 180.0, 200.0, 215.0, 230.0, 260.0]
[1.108496, 0.475858, 0.380635, 0.397509, 0.440991, 0.523009]
***** Ticker :  VRTX  Date :  6/04/21 ******
[105.0, 170.0, 192.5, 210.0, 227.5, 255.0, 300.0]
[1.088348, 0.408633, 0.412208, 0.37204, 0.471888, 0.671391, 0.67402]
***** Ticker :  VRTX  Date :  7/06/21 ******
[120.0, 175.0, 187.5, 200.0, 212.5, 225.0]
[0.878967, 0.348737, 0.277993, 0.269485, 0.328778, 0.385977]
***** Ticker :  VRTX  Date :  8/05/21 ******
[165.0, 185.0, 195.0, 205.0, 230.0, 245.0]
[0.34046, 0.270132, 0.237591, 0.186281, 0.328939, 0.502139]
***** Ticker :  VRTX  Date :  9/07/21 ******
[140.0, 160.0, 175.0, 190.0, 205.0, 220.0, 225.0

***** Ticker :  VZ  Date :  4/05/21 ******
[50.0, 55.0, 57.0, 59.0, 61.0, 63.0, 70.0]
[0.445342, 0.238114, 0.181231, 0.172849, 0.167374, 0.196567, 0.412177]
***** Ticker :  VZ  Date :  5/05/21 ******
[50.0, 55.0, 57.0, 59.0, 61.0, 63.0, 65.0]
[0.365403, 0.19112, 0.158055, 0.146295, 0.148774, 0.16498, 0.318849]
***** Ticker :  VZ  Date :  6/04/21 ******
[45.0, 53.0, 55.0, 57.0, 59.0, 61.0, 70.0]
[0.505751, 0.233326, 0.18745, 0.138527, 0.134872, 0.156251, 0.323217]
***** Ticker :  VZ  Date :  7/06/21 ******
[45.0, 51.0, 54.0, 56.0, 58.0, 60.0, 65.0]
[0.607983, 0.350347, 0.185248, 0.149503, 0.148996, 0.16999, 0.268243]
***** Ticker :  VZ  Date :  8/05/21 ******
[49.0, 52.0, 55.0, 57.0, 59.0, 65.0]
[0.206658, 0.160048, 0.128756, 0.123893, 0.144163, 0.363134]
***** Ticker :  VZ  Date :  9/07/21 ******
[45.0, 51.0, 53.0, 55.0, 57.0, 59.0, 65.0]
[0.23864, 0.226079, 0.140448, 0.122044, 0.132673, 0.193169, 0.274255]
***** Ticker :  VZ  Date :  10/07/21 ******
[45.0, 48.0, 51.0, 54.0, 57.0, 60.0

***** Ticker :  WMT  Date :  11/08/21 ******
[105.0, 141.0, 145.0, 149.0, 165.0, 180.0]
[0.576491, 0.23097, 0.216537, 0.208337, 0.232259, 0.384451]
***** Ticker :  WMT  Date :  12/08/21 ******
[100.0, 120.0, 130.0, 137.0, 144.0, 155.0, 200.0]
[0.668087, 0.373924, 0.230677, 0.207342, 0.209864, 0.256617, 0.702492]
***** Ticker :  WMT  Date :  1/07/22 ******
[100.0, 133.0, 139.0, 145.0, 155.0, 185.0, 195.0]
[0.627735, 0.252227, 0.203186, 0.176283, 0.181711, 0.361723, 0.406996]
***** Ticker :  WMT  Date :  2/07/22 ******
[100.0, 126.0, 132.0, 138.0, 144.0, 150.0, 195.0]
[0.468129, 0.301289, 0.263213, 0.243801, 0.237873, 0.237529, 0.504094]
***** Ticker :  WMT  Date :  3/09/22 ******
[100.0, 125.0, 132.0, 139.0, 146.0, 165.0, 195.0]
[0.68265, 0.335481, 0.25175, 0.237082, 0.220843, 0.287906, 0.503172]
***** Ticker :  WMT  Date :  4/08/22 ******
[125.0, 145.0, 148.0, 155.0, 170.0, 185.0, 195.0]
[0.396023, 0.216574, 0.223494, 0.192308, 0.205938, 0.298189, 0.365292]
***** Ticker :  WMT  Date : 

***** Ticker :  XOM  Date :  3/05/21 ******
[46.5, 55.0, 57.0, 61.0, 63.0, 70.0]
[0.412302, 0.38873, 0.382475, 0.376126, 0.376386, 0.40755]
***** Ticker :  XOM  Date :  4/05/21 ******
[45.0, 51.5, 54.0, 56.5, 59.0, 61.5, 70.0]
[0.520511, 0.338399, 0.325337, 0.30369, 0.311493, 0.325909, 0.572995]
***** Ticker :  XOM  Date :  5/05/21 ******
[50.5, 57.0, 59.0, 61.0, 64.0, 70.0]
[0.368035, 0.296498, 0.274432, 0.285043, 0.290577, 0.332952]
***** Ticker :  XOM  Date :  6/04/21 ******
[45.0, 56.5, 59.0, 61.5, 64.0, 66.5, 80.0]
[0.693548, 0.292838, 0.27723, 0.27135, 0.269693, 0.275392, 0.43846]
***** Ticker :  XOM  Date :  7/06/21 ******
[50.0, 55.0, 59.0, 61.5, 64.0, 66.5, 75.0]
[0.493354, 0.362531, 0.304512, 0.298514, 0.296243, 0.298326, 0.377103]
***** Ticker :  XOM  Date :  8/05/21 ******
[53.0, 55.0, 57.0, 59.0, 61.0, 70.0]
[0.262553, 0.265641, 0.2646, 0.26581, 0.273661, 0.36436]
***** Ticker :  XOM  Date :  9/07/21 ******
[40.0, 49.0, 52.0, 55.0, 58.0, 61.0, 70.0]
[0.619734, 0.342608, 0.

In [24]:
ticker_hash_cusip = {}
option_ticker_cusip = options_df[['ticker','cusip']]
option_ticker_cusip.drop_duplicates(subset = ['cusip'], keep = 'first', inplace = True) 

/var/folders/cp/p3rk934j69l8v61fbfgp2c5r0000gn/T/ipykernel_1680/3478128795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_ticker_cusip.drop_duplicates(subset = ['cusip'], keep = 'first', inplace = True)


In [25]:
intial_investment = 100
returns_hash = {}
filtered_tickers = {}
for i,date in enumerate(dates_to_run_monthly):

    highSkewed,lowSkewed,fwd_date,long_portfolio_return,long_portfolio_return2,short_portfolio_return,benchmark_portfolio_return = returns(date,intial_investment,moments_monthly_data,option_ticker_cusip)
    returns_hash[date] = [fwd_date,intial_investment,long_portfolio_return['CurrentValue'],long_portfolio_return2['CurrentValue'],short_portfolio_return['CurrentValue'],benchmark_portfolio_return['CurrentValue']]    
    filtered_tickers[date] = highSkewed+list(long_portfolio_return.loc[highSkewed].values)+lowSkewed+list(short_portfolio_return.loc[lowSkewed].values)

return_final_df = pd.DataFrame(returns_hash).T
return_final_df.columns = ['Future Date','Intial Invetment','Long Portfolio','Long Portfolio2','Short Portfolio','Benchmark Portfolio']
return_final_df['Percent'] = 100*(return_final_df['Long Portfolio'] - return_final_df['Benchmark Portfolio'])/(return_final_df['Intial Invetment']*10)

Short :
['ACN', 'NOW', 'MA', 'REGN', 'MMC']
Long : 
['TJX', 'DHR', 'SYK', 'BLK', 'NEE']
Short :
['CB', 'PANW', 'ACN', 'NEE', 'JPM']
Long : 
['ABBV', 'PLD', 'BKNG', 'HD', 'NFLX']
Short :
['ACN', 'JPM', 'QCOM', 'V', 'REGN']
Long : 
['BLK', 'GOOG', 'SPGI', 'BKNG', 'GOOGL']
Short :
['PANW', 'V', 'META', 'NOW', 'COST']
Long : 
['TSLA', 'MA', 'SYK', 'VRTX', 'CB']
Short :
['CVX', 'CRM', 'PM', 'NVDA', 'META']
Long : 
['NFLX', 'CB', 'MMC', 'VRTX', 'TSLA']
Short :
['MCD', 'NOW', 'ADBE', 'REGN', 'TSLA']
Long : 
['MA', 'NFLX', 'VRTX', 'COST', 'BKNG']
Short :
['CRM', 'SPGI', 'GS', 'MA', 'ACN']
Long : 
['MCD', 'CB', 'GOOGL', 'VRTX', 'GOOG']
Short :
['MCD', 'GS', 'REGN', 'NVDA', 'ADBE']
Long : 
['BLK', 'META', 'SPGI', 'TSLA', 'GOOGL']
Short :
['V', 'MCD', 'META', 'ACN', 'PANW']
Long : 
['TSLA', 'CB', 'SYK', 'GOOGL', 'MMC']
Short :
['META', 'MCD', 'MA', 'CRM', 'NOW']
Long : 
['SPGI', 'NEE', 'GOOG', 'BLK', 'BKNG']
Short :
['LIN', 'ACN', 'MA', 'PANW', 'CRM']
Long : 
['META', 'ADBE', 'REGN', 'GOOG', 'SPG

In [26]:
return_final_df

,Future Date,Intial Invetment,Long Portfolio,Long Portfolio2,Short Portfolio,Benchmark Portfolio,Percent
1/02/18,2/01/18,100,839.60009,419.800045,426.002131,956.848014,-11.724792
2/01/18,3/03/18,100,808.379464,404.189732,395.435027,897.780056,-8.940059
3/05/18,4/04/18,100,771.815969,385.907984,381.702996,867.531377,-9.571541
4/04/18,5/04/18,100,817.499524,408.749762,428.036841,941.998979,-12.449945
5/04/18,6/03/18,100,825.135911,412.567956,411.759168,928.642981,-10.350707
...,...,...,...,...,...,...,...
10/07/22,11/06/22,100,789.124466,394.562233,428.170984,935.988584,-14.686412
11/07/22,12/07/22,100,842.846301,421.42315,403.014443,931.726552,-8.888025
12/07/22,1/06/23,100,812.170373,406.085187,429.164499,937.684995,-12.551462
1/06/23,2/05/23,100,787.822659,393.91133,431.137247,931.66112,-14.383846


In [ ]:
long_columns = ["Long-"+str(i) for i in range(1,6)]
long_columns_1 = ["Long-"+str(i)+"Return" for i in range(1,6)]
short_columns = ["Short-"+str(i) for i in range(1,6)]
short_columns_2 = ["Short-"+str(i)+"Return" for i in range(1,6)]
filtered_tickers_df = pd.DataFrame(filtered_tickers).T
filtered_tickers_df.columns = long_columns+long_columns_1+short_columns+short_columns_2

In [ ]:
percent_plot = pd.DataFrame(return_final_df['Percent'])
percent_plot.index = pd.to_datetime(return_final_df.index,format="%m/%d/%y")
percent_plot.index = percent_plot.index.strftime('%b %Y')

In [ ]:

# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('Moments_Weekly_Duration30_MIDCAP_Combined_NEW.xlsx',engine='xlsxwriter')   
workbook=writer.book

moments_monthly_data['Date'] = pd.to_datetime(moments_monthly_data['Date'],format="%m/%d/%y")


return_final_df.index = pd.to_datetime(return_final_df.index,format="%m/%d/%y")
return_final_df['Future Date'] = pd.to_datetime(return_final_df['Future Date'],format="%m/%d/%y")

moments_monthly_data.to_excel(writer,sheet_name='Moments Monthly')
filtered_tickers_df.to_excel(writer,sheet_name='Tickers')
return_final_df.to_excel(writer,sheet_name='Returns')
writer.save()

In [ ]:
sp500_data = yf.download('^GSPC', start='2018-01-01', end='2022-12-31')

# Calculate monthly returns
monthly_returns = sp500_data['Adj Close'].resample('M').ffill().pct_change()
monthly_returns = monthly_returns.shift(-1)*100
monthly_returns.index = monthly_returns.index.strftime('%b %Y')
# Plotting
plt.figure(figsize=(20, 6))
plt.plot(monthly_returns.index, monthly_returns, color='blue', linestyle='-')

plt.plot(percent_plot['Percent'])
plt.legend(['S&P 500','Portfolio']) 
plt.title('S&P 500 Monthly Returns')
plt.xlabel('Date')
plt.ylabel('Monthly Return')
plt.grid(True)
plt.show()